## finetuning
### huggingface上にあるLLMをinstallしてきてfinetuning

In [2]:
## training dataの収納

import os
import pandas as pd
import json

# ファイルパスの設定

for dataset in ['train','val']:
    txt_folder_path = f"../radnlp_2024_train_val_20240731/ja/main_task/{dataset}/"  # TXTファイルが保存されているフォルダ
    csv_file_path = f"../radnlp_2024_train_val_20240731/ja/main_task/{dataset}/label.csv"  # ラベルファイル

    # ラベルCSVファイルを読み込む
    label_data = pd.read_csv(csv_file_path)

    # ファインチューニング用データを格納するリスト
    finetune_data = []

    # フォルダ内のTXTファイルを処理
    for txt_file in os.listdir(txt_folder_path):
        if txt_file.endswith(".txt"):
            file_id = txt_file.split(".")[0]  # ファイル名からIDを取得
            file_path = os.path.join(txt_folder_path, txt_file)

            # 対応するラベルを検索
            matching_label = label_data[label_data["id"] == int(file_id)]
            if not matching_label.empty:
                # テキストを読み込む
                with open(file_path, 'r', encoding='utf-8') as file:
                    content = file.read().strip()

                # ラベルを取得
                t = matching_label["t"].values[0]
                n = matching_label["n"].values[0]
                m = matching_label["m"].values[0]

                # ファインチューニング用データを追加
                finetune_data.append({
                    "prompt": content,
                    "completion": f"{t} {n} {m}"
                })

    # ファインチューニング用データをJSONL形式で保存
    output_file = f"../finetune_jsons/finetune_dataset_{dataset}.jsonl"
    with open(output_file, 'w', encoding='utf-8') as jsonl_file:
        for entry in finetune_data:
            json.dump(entry, jsonl_file, ensure_ascii=False)
            jsonl_file.write('\n')

    print(f"ファインチューニング用データが {output_file} に保存されました！")


ファインチューニング用データが ../finetune_jsons/finetune_dataset_train.jsonl に保存されました！
ファインチューニング用データが ../finetune_jsons/finetune_dataset_val.jsonl に保存されました！


In [ ]:
import os
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import load_dataset, Dataset

# GPUの確認
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# モデルとトークナイザーのロード
model_name = "pfnet/Llama3-Preferred-MedSwallow-70B"  # 他のモデル（例: "bigscience/bloom-560m"）に変更可能
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

# ファインチューニングデータの準備
def prepare_dataset(jsonl_file_path):
    """
    JSONL形式のファインチューニングデータをHugging Face Dataset形式に変換。
    """
    data = []
    with open(jsonl_file_path, 'r', encoding='utf-8') as jsonl_file:
        for line in jsonl_file:
            entry = json.loads(line.strip())
            data.append({
                "prompt": entry["prompt"],
                "completion": entry["completion"]
            })

    # Hugging Face Datasetに変換
    dataset = Dataset.from_list(data)
    return dataset

# データセットのロード
jsonl_file_path_train = "../finetune_jsons/finetune_dataset_train.jsonl"
dataset_train = prepare_dataset(jsonl_file_path_train)
jsonl_file_path_val = "../finetune_jsons/finetune_dataset_val.jsonl"
dataset_val = prepare_dataset(jsonl_file_path_val)

with open('../tnm_prompt.txt', 'r', encoding='utf-8') as file:
    tnm_prompt_text = file.read()
    
    
tnm_prompt_text_base = (
    "あなたは優秀な医師です。以下の文章に基づき肺癌に関して常に正しい判断ができます。"
    "進行度分類は以下のTNM第８版に準拠しています。何も言わずに以下を覚え、与えられた文章からよく考えてTNM分類を選んでください。\n\n"
)

tnm_prompt = (
    f"{tnm_prompt_text_base}\n\n"
    f"{tnm_prompt_text}\n\n"
    "以下の文章を読んで、TNM分類を正確に選択し、必ず以下の形式で出力してください：\n"
    "T<number>[optional_letter] N<number>[optional_letter] M<number>[optional_letter]\n\n"
)
print(tnm_prompt)
# データをトークナイズ
def preprocess_function(examples):
    """
    入力データをトークン化し、モデル形式に変換。
    """
    inputs = [f"{tnm_prompt}{prompt} 出力：" for prompt in examples['prompt']]
    targets = [completion for completion in examples['completion']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = dataset_train.map(preprocess_function, batched=True)
eval_dataset = dataset_val.map(preprocess_function, batched=True)


# トレーニング設定
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
    push_to_hub=False,
    fp16=torch.cuda.is_available()
)

# Trainerの定義
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)

# モデルのファインチューニング
trainer.train()

# ファインチューニング済みモデルの保存
trainer.save_model("../finetuned_model")
tokenizer.save_pretrained("../finetuned_model")

# テストデータを用いて予測
def predict(model, tokenizer, text):
    """
    テスト用データからモデルの予測を生成。
    """
    inputs = tokenizer(f"{text} 出力：", return_tensors="pt", max_length=512, truncation=True).to(device)
    outputs = model.generate(**inputs, max_length=128, num_return_sequences=1)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# テスト用の入力
test_prompt = "左肺門部に 37mm 大の腫瘤影を認め、ご指摘の肺癌が疑われます。\n縦隔に有意なリンパ節腫大は認めません。\n胸水はありません。\n背部皮下に腫瘤を認め、粉瘤などと思われます。"

# 予測を実行
predicted_output = predict(model, tokenizer, test_prompt)
print("予測結果:", predicted_output)


/home/jubuntu/anaconda3/lib/python3.8/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

2024-11-27 00:09:58.972643: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-27 00:09:59.533124: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/jubuntu/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Using device: cuda


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/765 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/59.6k [00:00<?, ?B/s]

model-00001-of-00030.safetensors:   0%|          | 0.00/4.58G [00:00<?, ?B/s]